# Generate cell line dictionary
We create a dictionary with the following keys, indexed by cell line CCLE name, for use with the Molecular Oncology Almanac. Specifically, this dictionary is used for quick look ups when generating the report. It is saved and read with pickle.

- `ccle_name`: string, the CCLE designated name of the cell line
- `broad_name`: string, the Broad / DepMap designated name of the cell line
- `sanger_name`: string, the Sanger Institute designated name of the cell line
- `tumor_type`: string, the tumor ontology from CCLE
- `events`: list, clinically relevant events labeled either as putative actionable or investigate actionability by MOAlmanac
- `all somatic variants`: list, all somatic variants observed in the cell line whose gene appears in either MOAlmanac, Cancer Hotspots, or CGC
- `all copy number alterations`: list, all copy number alterations observed in the cell line whose gene appears in either MOAlmanac, Cancer Hotspots, or CGC
- `all fusions`: list, all fusions observed in the cell line whose gene appears in either MOAlmanac, Cancer Hotspots, or CGC
- `sensitive to`: list, all therapy names that have a zscore of less than or equal to 2 for the cell line
- `zscores`: pandas dataframe, this is a subset of the `gdsc` file that contains model id, zscore, AUC, ln_ic50, and therapy name. 

Move this notebook to `moalmanac/moalmanac` to run.

In [ ]:
import glob as glob
import pandas as pd
import pickle

import datasources

dbs_preclinical = datasources.Preclinical.import_dbs()
dbs_preclinical.keys()

In [ ]:
variants = dbs_preclinical['variants']
cnas = dbs_preclinical['copy_number_alterations']
fusions = dbs_preclinical['fusions']
gdsc = dbs_preclinical['gdsc']

for column in ['ln_ic50', 'AUC', 'z_score']:
    gdsc[column] = gdsc[column].astype(float)
gdsc.sort_values('z_score', ascending=True, inplace=True)

summary = dbs_preclinical['summary']
summary['use'] = summary['use'].replace('True', 1).replace('False', 0)
summary = summary[summary['use'].eq(1)]

print(summary.columns)
summary.head()

In [ ]:
def display_feature_variants(dataframe):
    dataframe.reset_index(drop=True, inplace=True)
    strings = []
    for index in dataframe.index:
        series = dataframe.loc[index, :]
        if series.loc['feature_match_4'] == 1:
            strings.append(' '.join([series.loc['feature'], series.loc['alteration']]))
        elif series.loc['feature_match_3'] == 1:
            strings.append(' '.join([series.loc['feature'], series.loc['alteration_type']]))
        elif series.loc['feature_match_2'] == 1:
            strings.append(' '.join([series.loc['feature'], 'somatic variant']))
        else:
            continue
    return sorted(strings)

def display_feature_variants_all(dataframe):
    dataframe.reset_index(drop=True, inplace=True)
    dataframe['display'] = dataframe['feature'] + ' ' + dataframe['alteration']
    list_ = dataframe['display'].dropna().sort_values().tolist()
    return list_


def display_feature_cn(dataframe):
    dataframe.reset_index(drop=True, inplace=True)
    list_ = (dataframe['feature'] + ' ' + dataframe['alteration_type']).sort_values().dropna().tolist()
    return list_

def display_feature_fusion(dataframe):
    strings = []
    for index in dataframe.index:
        series = dataframe.loc[index, :]
        if int(series.loc['feature_match_4']) == 1:
            strings.append(series.loc['feature'] + '--' + series.loc['partner'])
        elif int(series.loc['feature_match_2']) == 1:
            strings.append(series.loc['which_match'] + ' Fusion')
        else:
            continue
    return sorted(strings)

def display_feature_fusion_all(dataframe):
    dataframe['display'] = dataframe['feature'] + '--' + dataframe['partner']
    return dataframe['display'].sort_values().dropna().tolist()

def subset_dataframe(dataframe):
    idx_almanac = dataframe['feature_match_1'].astype(int).eq(1)
    idx_hotspot = dataframe['cancerhotspots_bin'].astype(int).eq(1)
    idx_cgc = dataframe['cgc_bin'].astype(int).eq(1)
    return dataframe[idx_almanac | idx_hotspot | idx_cgc].reset_index(drop=True)

In [ ]:
dictionary = {}

for column in variants.columns[variants.columns.str.contains('feature_match')]:
    variants[column] = variants[column].astype(int)
    cnas[column] = cnas[column].astype(int)
    fusions[column] = fusions[column].astype(int)

for label, group in summary.groupby('ccle_name'):
    series = group.loc[group.index[0], :]
    
    broad = series.loc['broad']
    sanger = series.loc['sanger']
    tumor_type= series.loc['TUMOR_TYPE']
    
    label_variants = variants[variants['model_id'].eq(broad)].reset_index(drop=True)
    strings_variants = display_feature_variants(label_variants[label_variants['feature_match_2'].eq(1)])
    strings_variants_all = display_feature_variants_all(subset_dataframe(label_variants))
    
    label_cns = cnas[cnas['model_id'].eq(broad)].reset_index(drop=True)
    strings_cns = display_feature_cn(label_cns[label_cns['feature_match_2'].eq(1)])
    strings_cns_all = display_feature_cn(subset_dataframe(label_cns))
    
    label_fusions = fusions[fusions['model_id'].eq(broad)].reset_index(drop=True)
    strings_fusions = display_feature_fusion(label_fusions[label_fusions['feature_match_2'].eq(1)])
    strings_fusions_all = display_feature_fusion_all(subset_dataframe(label_fusions))
    
    clinically_relevant_events = sorted(strings_variants + strings_cns + strings_fusions)
    
    label_gdsc = gdsc[gdsc['model_id'].eq(broad)].reset_index(drop=True)
    label_gdsc['ccle_name'] = label
    sensitive_to = label_gdsc[label_gdsc['z_score'].le(-2)]['therapy_name'].sort_values().tolist()
    
    subdictionary = {}
    subdictionary['ccle_name'] = label
    subdictionary['broad_name'] = broad
    subdictionary['sanger_name'] = sanger
    subdictionary['tumor_type'] = tumor_type
    subdictionary['events'] = ', '.join(clinically_relevant_events)
    
    subdictionary['all somatic variants'] = ', '.join(strings_variants_all)
    subdictionary['all copy number alterations'] = ', '.join(strings_cns_all)
    subdictionary['all fusions'] = ', '.join(strings_fusions_all)
    
    subdictionary['sensitive to'] = ', '.join(sensitive_to)
    subdictionary['zscores'] = label_gdsc

    dictionary[broad] = subdictionary

In [ ]:
import pickle

output = open('cell-lines.pkl', 'wb')
pickle.dump(dictionary, output)
output.close()

In [ ]:
!cp cell-lines.pkl ~/Github/moalmanac-paper/analyses/preclinical/
!mv cell-lines.pkl datasources/preclinical/